# SI master
処置料情報を日付に対応したマスターとして作成する。
処置料は改定月の翌月2日から有効なものとなる。

In [1]:
import pandas as pd
import json
import pathlib
import datetime
import numpy as np

ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")
ts

'20231122_143531_738708'

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 250)

マスター読み込み関数  
マスターにはカラム名が無いので、c+番号でカラム名をつける

87, 88 is dateinfo

In [14]:
def get_file_df(filepath):
    df_i = pd.read_csv(filepath,encoding='cp932', header=None,low_memory=False,
                       dtype={86:str, 87:str, 33:str})
    
    df_i.columns = ["c"+ str(i) for i in range(len(df_i.columns))]
    df_i["filepath"] = f
    return df_i

In [15]:
# f_pk = "./masterfile_20230426_150151_224504.pkl"
# f_pk = "./masterfile_20230427_095353_581518.pkl"
# f_pk = "./masterfile_20230603_100705_814763.pkl"
f_pk = "./masterfile_20231122_100722_065215.pkl"
df_files = pd.read_pickle(f_pk)

df_files.head()

,filepath,filename,parent,gparent,bearfile,timeinfo,prefix,ymdinfo,postfix,is_kaitei,ymd,next_month_day1,next_month_day2,sha256,md5,size,lastfmodification,filecreation
0,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,05算定回数テーブル.csv,サンプル,osirase_04,05算定回数テーブル,,05算定回数テーブル,,,False,NaT,NaT,NaT,890542a2abd7d040ab78783bb838af1c456dcdd54e314e...,59b395aa7c467f838d818cd0bf2580cd,10.673828125kB,2023-11-22 10:07:26.207215,2023-11-22 10:07:26.207215
1,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,01補助マスターテーブル.csv,tensuhyo_02,tensuhyo_02_20120213,01補助マスターテーブル,,01補助マスターテーブル,,,False,NaT,NaT,NaT,38cfc25516491e252aef17f93ef4fa4de0ca64ee093485...,cf29e86e3c71970e1f4f88e1e74f5379,886.939453125kB,2023-11-22 10:07:26.234844,2023-11-22 10:07:26.234844
2,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,02包括テーブル.csv,tensuhyo_02,tensuhyo_02_20120213,02包括テーブル,,02包括テーブル,,,False,NaT,NaT,NaT,2b939f604846452eed5c891b4ad97d2b8d9633aa0d9f06...,70d068ccdf6093f9b24b963bb28062e1,6001.912109375kB,2023-11-22 10:07:26.317167,2023-11-22 10:07:26.317167
3,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,03-1背反テーブル1.csv,tensuhyo_02,tensuhyo_02_20120213,03-1背反テーブル1,1,03-1背反テーブル,1,,False,NaT,NaT,NaT,e9abb2b236605c230435dafe08e09bbf93941b402582a8...,9443610957dd09433753eb9230b2e1bf,578.05078125kB,2023-11-22 10:07:26.325180,2023-11-22 10:07:26.325180
4,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,03-2背反テーブル2.csv,tensuhyo_02,tensuhyo_02_20120213,03-2背反テーブル2,2,03-2背反テーブル,2,,False,NaT,NaT,NaT,92000a77113ec6044414d4f6a325a44463d850db653205...,fcb7bb8a12b2244dc42dffb0ed5ad644,392.81640625kB,2023-11-22 10:07:26.332179,2023-11-22 10:07:26.332179


処置マスターのみ

In [16]:
df_files_ =  df_files\
    .query("prefix == 's' or prefix == 's_ALL' ")\
    .drop_duplicates(subset=["filename", "sha256", "md5", "size"], keep='first', ) \
    .reset_index(drop=False)

In [17]:
# df_files_["prefix"].count()

df_dt from step 2

In [18]:
# fdt_pk = "./datefile_20230426_100605_859194.pkl"
# fdt_pk = "./datefile_20230427_095353_581518.pkl"
# fdt_pk = "./datefile_20230603_100705_814763.pkl"
fdt_pk = "./datefile_20231122_100722_065215.pkl"
dfdt = pd.read_pickle(fdt_pk)
dfdt.head()

,dt,dt_str,yy_str,mm_str,dd_str,is_past
0,2010-01-01,2010-01-01,2010,1,1,True
1,2010-01-02,2010-01-02,2010,1,2,True
2,2010-01-03,2010-01-03,2010,1,3,True
3,2010-01-04,2010-01-04,2010,1,4,True
4,2010-01-05,2010-01-05,2010,1,5,True


In [19]:
dt_start = dfdt["dt"].iat[0]
dt_start_str = dfdt["dt_str"].iat[0]
dt_start, dt_start_str

(Timestamp('2010-01-01 00:00:00'), '2010-01-01')

In [20]:
dt_end = dfdt["dt"].iat[-1]
dt_end_str = dfdt["dt_str"].iat[-1]
dt_end,dt_end_str

(Timestamp('2023-08-31 00:00:00'), '2023-08-31')

当該ファイル全部読み込み

In [21]:
%%time

l_df = []
for r in df_files_.itertuples():
    # print(r)
    f = r.filepath
    l_df.append(get_file_df(f))
# df_i.head()

CPU times: user 3.62 s, sys: 779 ms, total: 4.4 s
Wall time: 6.36 s


In [23]:
def pick_dt_si(c_change, c_enduse):
    l_res = []
    for s_change, s_enduse in zip(c_change, c_enduse):
        if s_change != "":
            l_res.append(s_change)
        elif s_enduse !="":
            l_res.append(s_enduse)
        else:
            l_res.append("")
    return l_res

In [65]:
df_whole = pd.concat(l_df)\
    .merge(df_files_, on="filepath") \
    .assign(c86 = lambda df:df.c86.fillna(""),
            c87 = lambda df:df.c87.fillna("").str.replace("99999999", "")
           )\
    .assign(ymdc = lambda df: pick_dt_si(df.c86, df.c87))\
    .assign(ymdinfo = lambda df: [r.ymdc if r.ymdc != "" else r.ymd for r in df.itertuples()]) \
    .assign(ymd = lambda df:pd.to_datetime(df.ymdinfo, errors="coerce"))\
    .sort_values(["c2", "ymd"])\
    .assign(next_month_day1 = lambda df:df.ymd + pd.offsets.MonthBegin(1)) \
    .assign(next_month_day2 = lambda df:df.next_month_day1 + datetime.timedelta(days =1))
    # .sort_values(["c2", "ymdinfo"])
df_whole.shape

(105098, 170)

In [66]:
df_whole.head().T

,6621,13398,17933,24680,31129
c0,0,5,0,5,0
c1,S,S,S,S,S
c2,111000110,111000110,111000110,111000110,111000110
c3,2.0,NaN,2.0,NaN,2.0
c4,初診,NaN,初診,NaN,初診
c5,4.0,NaN,4.0,NaN,4.0
c6,ｼｮｼﾝ,NaN,ｼｮｼﾝ,NaN,ｼｮｼﾝ
c7,0.0,NaN,0.0,NaN,0.0
c8,0.0,NaN,0.0,NaN,0.0
c9,NaN,NaN,NaN,NaN,NaN


c4には診療行為省略名称

In [67]:
df_whole\
    .query("c4 == c4").head().T.head(15)

,6621,17933,31129,38320,47755
c0,0,0,0,5,0
c1,S,S,S,S,S
c2,111000110,111000110,111000110,111000110,111000110
c3,2.0,2.0,2.0,3.0,3.0
c4,初診,初診,初診,初診料,初診料
c5,4.0,4.0,4.0,7.0,7.0
c6,ｼｮｼﾝ,ｼｮｼﾝ,ｼｮｼﾝ,ｼｮｼﾝﾘｮｳ,ｼｮｼﾝﾘｮｳ
c7,0.0,0.0,0.0,NaN,0.0
c8,0.0,0.0,0.0,NaN,0.0
c9,NaN,NaN,NaN,NaN,NaN


In [68]:
df_whole\
    .query("c4 == c4")\
    ["c2"].unique().size
    

11591

c10の意味はレセプト電算処理システムマスターファイル使用説明書を参照されたい。  
１：金額  
３：点数（プラス）  
４：購入価格（点数）  
５：％加算  
６：％減算  
７：減点診療行為  
８：点数（マイナス）  


In [69]:
df_whole \
    .groupby(["c10"])\
    ["c2"].count()

c10
1.0      307
3.0    61301
4.0       68
5.0      433
6.0      391
7.0       14
8.0      829
Name: c2, dtype: int64

In [70]:
df_whole \
    .groupby(["c12"])\
    ["c2"]\
    .count()\
    .reset_index()\
    .set_index("c12")

,c2
c12,
0.0,41097
1.0,15256
2.0,7031


In [71]:
df_whole\
    .groupby("c2")\
    ["c0"].count()\
    .reset_index() \
    .sort_values("c0", ascending = False)

,c2,c0
8447,190130210,22
8448,190130310,22
8821,190171910,22
10514,193009310,21
4331,150337910,21
...,...,...
4245,150329310,1
4130,150317310,1
4129,150317210,1
4016,150303810,1


In [72]:
df_whole\
    .groupby("c0")\
    ["c2"].count()\
    .reset_index() 
# \
#     .sort_values("c0", ascending = False)

,c0,c2
0,0,53639
1,3,9676
2,5,37874
3,9,3909


IYとの違いは
"c23", "c24"→,"c58", "c59"  
 "c27"　剤形→
 "c31" 変更年月日
 c88, c89
 
また、利用可能な最古のマスターである  
https://www.ssk.or.jp/seikyushiharai/tensuhyo/kihonmasta/h24/kihonmasta_01.files/s20120305.csv  
において、引き継ぎの記載で価格等がNAになっているものが存在する。  
現状このさらに以前のマスターは利用できない。  

平成２２年度版医科診療行為マスターが利用できればこれは埋めることができるが、現状は利用不能  
→なので、多くの価格は2014年マスターによる価格定義になっている。

In [73]:
df_whole \
    .sort_values(["c2", "ymd"]) \
    .query("c2 == 111000370").T

,0,6622,17934,17665,31130,38321,47756,80668,96364,57529,59794,62442,65130,70683
c0,5,0,0,5,0,5,0,0,0,5,5,5,5,0
c1,S,S,S,S,S,S,S,S,S,S,S,S,S,S
c2,111000370,111000370,111000370,111000370,111000370,111000370,111000370,111000370,111000370,111000370,111000370,111000370,111000370,111000370
c3,NaN,9.0,9.0,NaN,9.0,NaN,9.0,9.0,9.0,NaN,NaN,NaN,NaN,9.0
c4,NaN,初診（乳幼児）加算,初診（乳幼児）加算,NaN,初診（乳幼児）加算,乳幼児加算（初診）,乳幼児加算（初診）,乳幼児加算（初診）,乳幼児加算（初診）,NaN,NaN,NaN,NaN,乳幼児加算（初診）
c5,NaN,7.0,7.0,NaN,7.0,10.0,10.0,10.0,10.0,NaN,NaN,NaN,NaN,10.0
c6,NaN,ｼｮｼﾝｶｻﾝ,ｼｮｼﾝｶｻﾝ,NaN,ｼｮｼﾝｶｻﾝ,ﾆｭｳﾖｳｼﾞｶｻﾝ,ﾆｭｳﾖｳｼﾞｶｻﾝ,ﾆｭｳﾖｳｼﾞｶｻﾝ,ﾆｭｳﾖｳｼﾞｶｻﾝ,NaN,NaN,NaN,NaN,ﾆｭｳﾖｳｼﾞｶｻﾝ
c7,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
c8,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
c9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# <!-- 2 項番12「新又は現点数」の「点数識別」
# (1) 「1:金額」
# 項番12「新又は現点数」に、食事療養費及び生活療養費並びに標準負担額等の単位を「金額」
# で設定していることを表す。
# (2) 「3:点数(プラス)」
# 項番12「新又は現点数」に、医科点数表又は歯科点数表等に規定する診療行為の単位を「点数」
# で設定していることを表す。
# (3) 「4:購入価格(点数)」
# 生体移植における臓器等提供者の療養上の費用等、所定点数が確定していない診療行為であるこ
# とを表す。
# 項番12「新又は現点数」には、「0.00」を設定する。
# (参 考)
# 「点数識別」に「4:購入価格(点数)」を設定している診療行為の例
# ・生体皮膚移植(提供者の療養上の費用)加算
# ・生体部分肝移植術(提供者の療養上の費用)加算
# ・生体部分肺移植術(提供者の療養上の費用)加算
# ・生体部分小腸移植術(提供者の療養上の費用)加算
# ・生体腎移植術(提供者の療養上の費用)加算
# ・造血幹細胞移植(提供者の療養上の費用)加算
# ・治験分控除後包括点数
# ・公害補償法控除後包括点数
# ・他医療機関診療費
# ・歯科診療費
# (4) 「5:%加算」
# 医科点数表又は歯科点数表等に「所定点数の100分の○○に相当する点数を加算する。」と規
# 定する診療行為(注加算及び通則加算)の加算割合を表す。
# 項番12「新又は現点数」には、加算割合を百分率(○○/100)で設定する。
# (5) 「6:%減算」
# 医科点数表又は歯科点数表等に「所定点数の100分の○○に相当する点数により算定する」と
# 規定する診療行為の逓減割合を表す。
# 項番12「新又は現点数」には、減算割合を百分率(○○/100)で設定する。

# (6) 「7:減点診療行為」
# 特定入院料等に包括される診療行為のうち、治験及び公害補償法の対象となる診療行為の点数を
# 合算して減点するためのコードを表す。
# 項番12「新又は現点数」には、「0.00」を設定する。
# (参 考)
# 「点数識別」に「7:減点診療行為」を設定している診療行為の例
# ・包括点数の治験減点分
# ・包括点数の公害補償法減点分
# (7) 「8:点数(マイナス)」
# 厚生労働大臣が定める基準を満たしていない場合等に、所定点数を減点する診療行為であること
# を表す。
# 項番12「新又は現点数」には、減算する「点数」を設定する。 -->

診療行為についてはc10で掛けたりはしていない。
c10=pointlabel=点数識別を付与しておくので、目的に応じて後で処理すること。

In [75]:
def c10c11fix(c10, c11):
    if c10 != c10 or c11 != c11:
        # c10 or c11がNAのならC11はそのまま
        return c11
    elif c11 ==0:
        return c11
    else:
        if c10 in [1,3,4,5,6,7,8]:
            mul = 1
        # elif c10 == 3:
        #     mul = 10
        # elif c10 == 4:
        #     mul = 0
        # elif c10 == 5:
        #     mul = 1
        # elif c10 ==6:
        #     mul = -1
        # elif c10 ==7:
        #     mul = -1
        # elif c10==8:
        #     mul = -10
        else:
            #rint(c10, c11)
            mul=1
        return mul*c11

In [76]:
# .query("c10 != 7 and c10 != 3") \していない。点数もマスターとして作成するため
# c11fillがNAのものは、ようするに2010年マスターを引き継いでいて点数などがわからない。除外する。
# s1>s2も除外する。これは直近のマスターで情報があった項目で、翌月2日よりまえにdt_endが存在する場合である。
df_sub = df_whole\
    .loc[:, ["c0", "c2", "c4", "c9", "c10", "c11", "c16","c20",
             "c58", "c59",
             "c90","c91","c92", "c93", "c94",# "c95", "c96", "c97", "c98","c99" ,
             "c88","c99",
             "ymd", "next_month_day1", "next_month_day2", "filename"]].copy()\
    .sort_values(["c2", "ymd"]) \
    .assign(c11fill_ = lambda df:df.groupby("c2")["c11"].fillna(method="ffill"))\
    .assign(c11fill  = lambda df:[c10c11fix(r.c10, r.c11fill_) for r in df.itertuples()])\
    .assign(c59fill_ = lambda df:df.groupby("c2")["c59"].fillna(method="ffill"))\
    .assign(c59fill  = lambda df:[c10c11fix(r.c58, r.c59fill_) for r in df.itertuples()])\
    .query("c11fill == c11fill")\
    .assign(s0 = lambda df:df.groupby("c2")["next_month_day2"].shift(1))\
    .assign(s1 = lambda df:df.next_month_day2)\
    .assign(s2 = lambda df:df.groupby("c2")["next_month_day2"].shift(-1))\
    .assign(dt_rank = lambda df:df.groupby("c2")["ymd"].rank(method='min'))\
    .assign(is_first_in_master = lambda df:np.isnan(df.s0))\
    .assign(is_last_in_master = lambda df:np.isnan(df.s2))\
    .assign(s0 = lambda df:df.s0.fillna(dt_start))\
    .assign(s2 = lambda df:df.s2.fillna(dt_end))\
    .query("s1 <= s2")
df_sub.head(5)

/tmp/ipykernel_63791/3757862757.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .assign(c11fill_ = lambda df:df.groupby("c2")["c11"].fillna(method="ffill"))\
/tmp/ipykernel_63791/3757862757.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .assign(c59fill_ = lambda df:df.groupby("c2")["c59"].fillna(method="ffill"))\


,c0,c2,c4,c9,c10,c11,c16,c20,c58,c59,c90,c91,c92,c93,c94,c88,c99,ymd,next_month_day1,next_month_day2,filename,c11fill_,c11fill,c59fill_,c59fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
6621,0,111000110,初診,NaN,3.0,270.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,240000.0,0.0,2012-04-01,2012-05-01,2012-05-02,s_ALL20140110.csv,270.0,270.0,0.0,0.0,2010-01-01,2012-05-02,2014-04-02,1.0,True,False
13398,5,111000110,NaN,NaN,NaN,282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-05,2014-04-01,2014-04-02,s_20140305.csv,282.0,282.0,0.0,0.0,2012-05-02,2014-04-02,2014-05-02,2.0,False,False
17933,0,111000110,初診,NaN,3.0,282.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,230000.0,0.0,2014-04-01,2014-05-01,2014-05-02,s_ALL20151111.csv,282.0,282.0,0.0,0.0,2014-04-02,2014-05-02,2016-04-02,3.0,False,False
24680,5,111000110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,NaN,NaN,NaN,2016-03-04,2016-04-01,2016-04-02,s_20160304.csv,282.0,282.0,0.0,0.0,2014-05-02,2016-04-02,2016-05-02,4.0,False,False
31129,0,111000110,初診,NaN,3.0,282.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,450000.0,0.0,2016-04-01,2016-05-01,2016-05-02,s_ALL20180221.csv,282.0,282.0,0.0,0.0,2016-04-02,2016-05-02,2018-04-02,5.0,False,False


In [77]:
df_sub.query("s1 > s2")

,c0,c2,c4,c9,c10,c11,c16,c20,c58,c59,c90,c91,c92,c93,c94,c88,c99,ymd,next_month_day1,next_month_day2,filename,c11fill_,c11fill,c59fill_,c59fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master


## bsl: Before SaLe  
上市前。診療行為を上市というのはおかしいが、ともかく新規にマスターとして登録される前の期間。DB上、上市前に当該コードが存在するかは不明である。なので最も古い価格を与える。
## bdm: Before Degital Master  
最古のデジタルマスターより前。デジタルマスターではそれ以前の値は不明である。なので最も古い価格を与える。  
## ws: Within Sale  
マスターに採用されている期間中。最新の価格改定は翌月２日に価格される。  
## aes: After End of Sale
廃止終了後。DB上、廃止後に当該コードが存在するかは不明である。なので最も新しい価格を与える。

In [78]:
def get_1st_tag (c0):
    if c0 == 3:
        return "bsl"
    elif c0 == 5:
        return "bdm"
    elif c0 == 0:
        return "bdm"
    elif c0 == 9:
        return "bdm"
    else:
        return "unknown"

c0の意味は  
０：前マスターの内容と同じであることを表す。  
１：抹消  
３：新規  
５：変更  
９：廃止  


In [79]:
d_c10 = df_sub \
    .query("c10 == c10")\
    .groupby(["c2"])\
    ["c10"].unique().to_dict()

for k, v in d_c10.items():
    if len(v) != 1:
        print(k)

In [80]:
df_sub\
    .query("c10 == c10")\
    .groupby("c10")\
    ["c2"].count()

c10
1.0      307
3.0    61286
4.0       68
5.0      433
6.0      391
7.0       14
8.0      829
Name: c2, dtype: int64

上記から、c2にたいしてc10は一意に規定されている様子  
また殆どは点数。


１：金額  
３：点数（プラス）  
４：購入価格（点数）  
５：％加算  
６：％減算  
７：減点診療行為  
８：点数（マイナス）  

新規。上市開始。

In [81]:
df_sub0 = df_sub.query("dt_rank == 1 and c0 ==3")\
    .rename(columns={
    "c10":"pointlab",
    "c11":"point",
    "s0":"t1",
    "s1":"t2"
}) \
    .loc[:,["c0", "c2","t1", "t2","pointlab", "point", "ymd"]]\
    .assign(tag = "bsl",
            ind = 0)
df_sub0.head(5)

,c0,c2,t1,t2,pointlab,point,ymd,tag,ind
6532,3,111012510,2010-01-01,2013-04-02,3.0,200.0,2013-03-11,bsl,0
6533,3,111012610,2010-01-01,2013-04-02,3.0,100.0,2013-03-11,bsl,0
17680,3,111012710,2010-01-01,2015-01-02,3.0,209.0,2014-12-12,bsl,0
24685,3,111012810,2010-01-01,2016-04-02,3.0,104.0,2016-03-04,bsl,0
38341,3,111012970,2010-01-01,2018-04-02,3.0,200.0,2018-03-05,bsl,0


初出だが新規ではない。  
医薬品では「前回の価格情報」を利用しているが、診療行為はこれが手に入らないので、bdmかつc11fillがここに入る。

In [82]:
df_sub1 = df_sub\
    .query("dt_rank == 1 and c0 !=3")\
    .rename(columns={
    "c10":"pointlab",
    "c11fill":"point",
    "s0":"t1",
    "s1":"t2"
})  \
   .loc[:,["c0", "c2","t1", "t2","pointlab", "point", "ymd"]]\
    .assign(tag = "bdm",
            ind = 0)
df_sub1.head(5)

,c0,c2,t1,t2,pointlab,point,ymd,tag,ind
6621,0,111000110,2010-01-01,2012-05-02,3.0,270.0,2012-04-01,bdm,0
6622,0,111000370,2010-01-01,2012-05-02,3.0,75.0,2012-04-01,bdm,0
6623,0,111000470,2010-01-01,2012-05-02,3.0,130.0,2012-04-01,bdm,0
6624,0,111000570,2010-01-01,2012-05-02,3.0,85.0,2012-04-01,bdm,0
6625,0,111000670,2010-01-01,2012-05-02,3.0,250.0,2012-04-01,bdm,0


終了ではない

In [83]:
df_sub2 = df_sub \
    .query("c0 != 9")\
    .rename(columns={
    "c10":"pointlab",
    "c11fill":"point",
    "s1":"t1",
    "s2":"t2"
}) \
    .loc[:,["c0","c2","t1", "t2","pointlab", "point", "ymd"]] \
    .assign(tag = "ws",
            ind = 1)
df_sub2.head()

,c0,c2,t1,t2,pointlab,point,ymd,tag,ind
6621,0,111000110,2012-05-02,2014-04-02,3.0,270.0,2012-04-01,ws,1
13398,5,111000110,2014-04-02,2014-05-02,NaN,282.0,2014-03-05,ws,1
17933,0,111000110,2014-05-02,2016-04-02,3.0,282.0,2014-04-01,ws,1
24680,5,111000110,2016-04-02,2016-05-02,NaN,282.0,2016-03-04,ws,1
31129,0,111000110,2016-05-02,2018-04-02,3.0,282.0,2016-04-01,ws,1


廃止

In [84]:
df_sub3 = df_sub \
    .query("c0 == 9")\
    .rename(columns={
    "c11fill":"point",
    "c10":"pointlab",
    "s1":"t1",
    "s2":"t2"
}) \
    .loc[:,["c0","c2","t1", "t2", "point", "ymd"]] \
    .assign(tag = "aes", ind = 2)
df_sub3.head()

,c0,c2,t1,t2,point,ymd,tag,ind
47509,9,111012970,2019-02-02,2023-08-31,200.0,2019-01-01,aes,2
47510,9,111013070,2019-02-02,2023-08-31,365.0,2019-01-01,aes,2
47511,9,111013170,2019-02-02,2023-08-31,695.0,2019-01-01,aes,2
47512,9,111013270,2019-02-02,2023-08-31,345.0,2019-01-01,aes,2
47513,9,111013370,2019-02-02,2023-08-31,75.0,2019-01-01,aes,2


In [85]:
df_sub\
    .query("c2 ==111000110 ")

,c0,c2,c4,c9,c10,c11,c16,c20,c58,c59,c90,c91,c92,c93,c94,c88,c99,ymd,next_month_day1,next_month_day2,filename,c11fill_,c11fill,c59fill_,c59fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
6621,0,111000110,初診,NaN,3.0,270.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,240000.0,0.0,2012-04-01,2012-05-01,2012-05-02,s_ALL20140110.csv,270.0,270.0,0.0,0.0,2010-01-01,2012-05-02,2014-04-02,1.0,True,False
13398,5,111000110,NaN,NaN,NaN,282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-05,2014-04-01,2014-04-02,s_20140305.csv,282.0,282.0,0.0,0.0,2012-05-02,2014-04-02,2014-05-02,2.0,False,False
17933,0,111000110,初診,NaN,3.0,282.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,230000.0,0.0,2014-04-01,2014-05-01,2014-05-02,s_ALL20151111.csv,282.0,282.0,0.0,0.0,2014-04-02,2014-05-02,2016-04-02,3.0,False,False
24680,5,111000110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,NaN,NaN,NaN,2016-03-04,2016-04-01,2016-04-02,s_20160304.csv,282.0,282.0,0.0,0.0,2014-05-02,2016-04-02,2016-05-02,4.0,False,False
31129,0,111000110,初診,NaN,3.0,282.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,450000.0,0.0,2016-04-01,2016-05-01,2016-05-02,s_ALL20180221.csv,282.0,282.0,0.0,0.0,2016-04-02,2016-05-02,2018-04-02,5.0,False,False
38320,5,111000110,初診料,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-05,2018-04-01,2018-04-02,s_20180305.csv,282.0,282.0,1.0,1.0,2016-05-02,2018-04-02,2018-05-02,6.0,False,False
47755,0,111000110,初診料,NaN,3.0,282.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,610000.0,0.0,2018-04-01,2018-05-01,2018-05-02,s_ALL20190906.csv,282.0,282.0,1.0,1.0,2018-04-02,2018-05-02,2019-09-02,7.0,False,False
79784,5,111000110,NaN,NaN,NaN,288.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-08-19,2019-09-01,2019-09-02,s_20190819.csv,288.0,288.0,1.0,1.0,2018-05-02,2019-09-02,2019-11-02,8.0,False,False
80667,0,111000110,初診料,NaN,3.0,288.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,610000.0,0.0,2019-10-01,2019-11-01,2019-11-02,s_ALL20200306.csv,288.0,288.0,0.0,0.0,2019-09-02,2019-11-02,2020-05-02,9.0,False,False
96363,0,111000110,初診料,NaN,3.0,288.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,640000.0,0.0,2020-04-01,2020-05-01,2020-05-02,s_ALL20220218.csv,288.0,288.0,0.0,0.0,2019-11-02,2020-05-02,2022-05-02,10.0,False,False


In [90]:
df_whole.query("c2 == 150007770").T

,524,7547,18947,32315,47383,49016,81928,97753,72292
c0,5,0,0,0,5,0,0,0,0
c1,S,S,S,S,S,S,S,S,S
c2,150007770,150007770,150007770,150007770,150007770,150007770,150007770,150007770,150007770
c3,20.0,20.0,20.0,20.0,NaN,20.0,20.0,20.0,20.0
c4,生体皮膚移植（提供者の療養上の費用）加算,生体皮膚移植（提供者の療養上の費用）加算,生体皮膚移植（提供者の療養上の費用）加算,生体皮膚移植（提供者の療養上の費用）加算,NaN,生体皮膚移植（提供者の療養上の費用）加算,生体皮膚移植（提供者の療養上の費用）加算,生体皮膚移植（提供者の療養上の費用）加算,生体皮膚移植（提供者の療養上の費用）加算
c5,NaN,10.0,10.0,10.0,13.0,13.0,13.0,13.0,13.0
c6,NaN,ｾｲﾀｲﾋﾌｲｼｮｸ,ｾｲﾀｲﾋﾌｲｼｮｸ,ｾｲﾀｲﾋﾌｲｼｮｸ,ｾｲﾀｲﾋﾌｲｼｮｸｶｻﾝ,ｾｲﾀｲﾋﾌｲｼｮｸｶｻﾝ,ｾｲﾀｲﾋﾌｲｼｮｸｶｻﾝ,ｾｲﾀｲﾋﾌｲｼｮｸｶｻﾝ,ｾｲﾀｲﾋﾌｲｼｮｸｶｻﾝ
c7,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
c8,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
c9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
df_span = pd.concat([df_sub0, df_sub1, df_sub2, df_sub3])\
    .sort_values(["c2", "ymd", "ind"])\
    .query("t1 != t2")
df_span.query("c2 == 150007770").head(50)

,c0,c2,t1,t2,pointlab,point,ymd,tag,ind
7547,0,150007770,2010-01-01,2012-05-02,4.0,0.0,2012-04-01,bdm,0
7547,0,150007770,2012-05-02,2014-05-02,4.0,0.0,2012-04-01,ws,1
18947,0,150007770,2014-05-02,2016-05-02,4.0,0.0,2014-04-01,ws,1
32315,0,150007770,2016-05-02,2018-05-02,4.0,0.0,2016-04-01,ws,1
49016,0,150007770,2018-05-02,2019-11-02,4.0,0.0,2018-04-01,ws,1
81928,0,150007770,2019-11-02,2020-05-02,4.0,0.0,2019-10-01,ws,1
97753,0,150007770,2020-05-02,2022-05-02,4.0,0.0,2020-04-01,ws,1
72292,0,150007770,2022-05-02,2023-08-31,4.0,0.0,2022-04-01,ws,1


In [108]:
dfdt.head()

,dt,dt_str,yy_str,mm_str,dd_str,is_past
0,2010-01-01,2010-01-01,2010,1,1,True
1,2010-01-02,2010-01-02,2010,1,2,True
2,2010-01-03,2010-01-03,2010,1,3,True
3,2010-01-04,2010-01-04,2010,1,4,True
4,2010-01-05,2010-01-05,2010,1,5,True


この処理が重い。全く美しくない。

In [93]:
# df_whole.groupby("c2")["ymd"].min().to_list()

In [109]:
%%time
# .assign(c13 = lambda df:df.groupby("c2")["c13"].fillna(method ="ffill"))\
    # .assign(c14 = lambda df:df.groupby("c2")["c14"].fillna(method ="ffill"))\
    # .assign(c15 = lambda df:df.groupby("c2")["c15"].fillna(method ="ffill"))\
    # .assign(c16 = lambda df:df.groupby("c2")["c16"].fillna(method ="ffill"))\
    # .assign(c31 = lambda df:df.groupby("c2")["c31"].fillna(method ="ffill"))\

df_meta = df_whole \
    .loc[:, ["c0", "c2", "c4", "c9", "c10",
             "c13", "c14","c15", "c16", "c31",
              "c89", "c90","c91","c92", "c93", "c94",
             "ymd", "next_month_day1", "next_month_day2", "filename"]].copy()\
    .sort_values(["c2", "ymd","filename"], ascending=[True, True, False]) \
    .assign(c4 = lambda df:df.groupby("c2")["c4"].fillna(method ="ffill"))\
    .assign(c9 = lambda df:df.groupby("c2")["c9"].fillna(method ="ffill"))\
    .assign(c10 = lambda df:df.groupby("c2")["c10"].fillna(method ="ffill"))\
    .assign(c89 = lambda df:df.groupby("c2")["c89"].fillna(method ="ffill"))\
    .assign(c90 = lambda df:df.groupby("c2")["c90"].fillna(method ="ffill"))\
    .assign(c91 = lambda df:df.groupby("c2")["c91"].fillna(method ="ffill"))\
    .assign(c92 = lambda df:df.groupby("c2")["c92"].fillna(method ="ffill"))\
    .assign(c93 = lambda df:df.groupby("c2")["c93"].fillna(method ="ffill"))\
    .assign(c94 = lambda df:df.groupby("c2")["c94"].fillna(method ="ffill"))\
    .query("c10 == c10")\
    .assign(dt_rank_neg = lambda df:df.groupby("c2")["ymd"].rank(ascending = False,method='first'))\
    .query("dt_rank_neg < 2") \
    .assign(c89 = lambda df:df.c89.astype(int))\
    .assign(c90 = lambda df:df.c90.astype(int))\
    .assign(c91 = lambda df:df.c91.astype(int))\
    .assign(c92 = lambda df:df.c92.astype(int))\
    .assign(c93 = lambda df:df.c93.astype(int))\
    .assign(c94 = lambda df:df.c94.astype(int))
    
df_meta.head(50)

<timed exec>:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:18: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future

CPU times: user 52.9 s, sys: 982 ms, total: 53.9 s
Wall time: 53.6 s


,c0,c2,c4,c9,c10,c13,c14,c15,c16,c31,c89,c90,c91,c92,c93,c94,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
70682,0,111000110,初診料,NaN,3.0,0.0,110.0,0.0,0.0,99999999.0,1,1,0,0,1,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70683,0,111000370,乳幼児加算（初診）,NaN,3.0,1.0,110.0,0.0,0.0,99999999.0,1,1,0,0,5,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70684,0,111000470,乳幼児育児栄養指導料,NaN,3.0,1.0,130.0,0.0,0.0,99999999.0,2,1,1,2,6,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70685,0,111000570,時間外加算（初診）,NaN,3.0,0.0,110.0,0.0,0.0,99999999.0,1,1,0,0,6,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70686,0,111000670,休日加算（初診）,NaN,3.0,0.0,110.0,0.0,0.0,99999999.0,1,1,0,0,6,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70687,0,111000770,深夜加算（初診）,NaN,3.0,0.0,110.0,0.0,0.0,99999999.0,1,1,0,0,6,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70688,0,111000870,時間外特例医療機関加算（初診）,NaN,3.0,0.0,110.0,0.0,0.0,99999999.0,1,1,0,0,6,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70689,0,111010770,乳幼児時間外特例医療機関加算（初診）（小児科外来診療料）,NaN,3.0,1.0,130.0,0.0,0.0,99999999.0,2,1,1,2,3,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70690,0,111011570,乳幼児夜間加算（小児科初診）,NaN,3.0,1.0,110.0,0.0,0.0,99999999.0,1,1,0,0,7,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
70691,0,111011670,乳幼児休日加算（小児科初診）,NaN,3.0,1.0,110.0,0.0,0.0,99999999.0,1,1,0,0,7,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0


In [110]:
df_meta\
    .query('c4.str.contains("肝切除")', engine='python')\
    .head()

,c0,c2,c4,c9,c10,c13,c14,c15,c16,c31,c89,c90,c91,c92,c93,c94,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
74390,0,150324010,胆嚢悪性腫瘍手術（肝切除（葉以上）を伴う）,NaN,3.0,0.0,500.0,0.0,0.0,99999999.0,2,10,675,0,3,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
74392,0,150324210,胆嚢悪性腫瘍手術（膵頭十二指腸切除及び肝切除（葉以上）を伴う）,NaN,3.0,0.0,500.0,0.0,0.0,99999999.0,2,10,675,0,5,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
89154,9,150348010,腹腔鏡下肝切除術（部分切除）,NaN,3.0,NaN,NaN,NaN,NaN,NaN,2,10,695,2,1,0,2020-03-31,2020-04-01,2020-04-02,s_20200305.csv,1.0
74548,0,150348110,腹腔鏡下肝切除術（外側区域切除）,NaN,3.0,0.0,500.0,0.0,0.0,99999999.0,2,10,695,2,2,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0
74675,0,150362210,胆嚢悪性腫瘍手術（肝切除（亜区域切除以上））,NaN,3.0,0.0,500.0,0.0,0.0,99999999.0,2,10,675,0,2,0,2022-04-01,2022-05-01,2022-05-02,s_ALL20231101.csv,1.0


In [111]:
df_meta \
    .query("c2 ==150443250")

,c0,c2,c4,c9,c10,c13,c14,c15,c16,c31,c89,c90,c91,c92,c93,c94,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
70476,5,150443250,ラジオ波焼灼療法（２ｃｍ以内）（腹腔鏡）,NaN,3.0,NaN,NaN,NaN,NaN,NaN,2,10,697,3,3,0,2023-06-01,2023-07-01,2023-07-02,s_20230601.csv,1.0


In [97]:
# df_sub.query("c2 == 111000110")

In [112]:
df_sub["c2"].unique().size

11591

In [113]:
[c2 for c2 in list(df_sub["c2"].unique()) if c2 not in df_meta["c2"].to_list()]

[]

In [114]:
102 % 100

2

In [120]:
d_output = "../output_si/"
outputtag = "sicode_"
# postfix = ".json.gz"
postfix = ".json"
pf = d_output + outputtag + "{}" + "_" + ts + postfix
pf

'../output_si/sicode_{}_20231122_143531_738708.json'

In [121]:
import json
# import gzip
import bz2
import joblib
# with gzip.open(write_file, 'wt', encoding="ascii") as zipfile:
#        json.dump(data, zipfile)
        
def d2j(d, f_json):
    with open(f_json,  mode='wt', encoding='utf-8') as file:
        json.dump(d, file, ensure_ascii=False, indent=2)

# def d2jz(d, f_json):
#     # with gzip.open(f_json,  mode='wt', encoding='utf-8') as zipfile:
#     with bz2.open(f_json,  mode='wt', encoding='utf-8') as zipfile:
#         json.dump(d, zipfile, ensure_ascii=False, indent=2)


いまさらだが書き出しは並列でやったほうが早い。  
当初ベタなループであったが、並列処理に修正。  
コアもりもりの計算機ならすぐ終わるはず。

ctn: code table number:コード表用番号  
マスターカラム90-94  
c89-93  

In [122]:
def c2j(c2):
    df_iter = dfdt.copy()\
        .loc[:,["dt","dt_str"]]\
        .assign(c2 = c2)\
        .merge(df_span, on= "c2")\
        .query("t1 <= dt and dt < t2")
    
    d_meta_sub = df_meta\
    .query("c2 == @c2").to_dict('records')[0]

    f_out = pf.format(c2)
    # print(f_out)
    d_i = {
        "sicode":int(c2),
        "name":d_meta_sub["c4"],
        "pointlabel":d_meta_sub["c10"],
        "ctn_1":d_meta_sub["c89"],
        "ctn_2":d_meta_sub["c90"],
        "ctn_3":d_meta_sub["c91"],
        "ctn_4":d_meta_sub["c92"],
        "ctn_5":d_meta_sub["c93"],
        "dt_start": dt_start_str,
        "dt_end":dt_end_str,
        "dt":dict()
    }
    
    for r in df_iter.itertuples():
        d_i["dt"][r.dt_str] = {"tag":r.tag,
                              "point":r.point}
    
    # print(d_i)
    d2j(d=d_i, f_json=f_out)
    
    return f_out

c2list = list(df_sub["c2"].unique())
len(c2list)

11591

In [124]:
%%time
result = joblib.Parallel(n_jobs=-5,verbose=2)(joblib.delayed(c2j)(i) for i in c2list)

[Parallel(n_jobs=-5)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-5)]: Done  33 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-5)]: Done 154 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-5)]: Done 357 tasks      | elapsed:   50.9s
[Parallel(n_jobs=-5)]: Done 640 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-5)]: Done 1005 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-5)]: Done 1450 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-5)]: Done 1977 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-5)]: Done 2584 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-5)]: Done 3273 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-5)]: Done 4042 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-5)]: Done 4893 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-5)]: Done 5824 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-5)]: Done 6837 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-5)]: Done 7930 tasks      | elapsed: 17.3min
[Parallel(n_jobs=-5)]: Done 9105 tasks      | 

CPU times: user 6min 38s, sys: 47.3 s, total: 7min 25s
Wall time: 23min 15s


In [125]:
f_meta_x = "../appendix/dfmeta_si_" + ts + ".xlsx"
f_meta_pk = "../appendix/dfmeta_si_" + ts + ".pkl"

df_meta.to_excel(f_meta_x)
df_meta.to_pickle(f_meta_pk)
f_meta_x, f_meta_pk

('../appendix/dfmeta_si_20231122_143531_738708.xlsx',
 '../appendix/dfmeta_si_20231122_143531_738708.pkl')

In [126]:
f_span_x = "../appendix/dfspan_si_" + ts + ".xlsx"
f_span_pk = "../appendix/dfspan_si_" + ts + ".pkl"
df_span.to_excel(f_span_x)
df_span.to_pickle(f_span_pk)
f_span_x, f_span_pk

('../appendix/dfspan_si_20231122_143531_738708.xlsx',
 '../appendix/dfspan_si_20231122_143531_738708.pkl')

In [48]:
df_sub\
    .query("c2 ==150153910")

,c0,c2,c4,c9,c10,c11,c16,c20,c58,c59,c90,c91,c92,c93,c94,c88,c99,ymd,next_month_day1,next_month_day2,filename,c11fill_,c11fill,c59fill_,c59fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
8615,0,150153910,経皮的冠動脈形成術,NaN,3.0,22000.0,0.0,3.0,0.0,0.0,10.0,546.0,0.0,0.0,0.0,57260000.0,0.0,2014-01-10,2014-02-01,2014-02-02,s_ALL20140110.csv,22000.0,22000.0,0.0,0.0,2010-01-01,2014-02-02,2014-04-02,1.0,True,False
13739,9,150153910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-05,2014-04-01,2014-04-02,s_20140305.csv,22000.0,22000.0,0.0,0.0,2014-02-02,2014-04-02,2023-06-03,2.0,False,True
